In [1]:
#### Install Packages ####
from requests import get
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import unicodedata
import pandas as pd
import datetime
from datetime import datetime
import requests
import re
import math
import numpy as np

In [2]:
###### Team Short Name #####
def team_three_letter_name(teamName):
    
    try:
        team_dict = {"Hawks":"ATL", "Celtics":"BOS", "Bulls":"CHI", "Cavaliers":"CLE", "Clippers": "LAC", "Hornets": "CHA", "Mavericks":"DAL", 
                  "Nuggets": "DEN", "Pistons":"DET", "Warriors":"GSW", "Rockets":"HOU", "Lakers": "LAL", "TimberWolves":"MIN", "Pacers": "IND",
                  "Grizzlies": "MEM", "Bucks": "MIL", "Heat":"MIA", "Knicks": "NYK", "Pelicans": "NOH", "Magic": "ORL", "Thunder": "OKC", 
                  "76ers":"PHI", "Blazers" :"POR", "Kings": "SAC", "Raptors": "TOR", "Nets": "BRK", "Spurs": "SAS", "Jazz": "UTA",
                  "Suns":"PHO","Wizards":"WAS", "Atlanta Hawks":"ATL", "Boston Celtics":"BOS", "Chicago Bulls":"CHI", "Cleveland Cavaliers":"CLE", 
                  "Los Angeles Clippers": "LAC", "Charlotte Hornets": "CHA", "Dallas Mavericks":"DAL", 
                  "Denver Nuggets": "DEN", "Detroit Pistons":"DET", "Golden State Warriors":"GSW", "Houston Rockets":"HOU", 
                  "Los Angeles Lakers": "LAL", "Minnesota TimberWolves":"MIN", "Indiana Pacers": "IND",
                  "Memphis Grizzlies": "MEM", "Milwaukee Bucks": "MIL", "Miami Heat":"MIA", "New York Knicks": "NYK", "New Orleans Pelicans": "NOH", 
                  "Orlando Magic": "ORL", "Oklahoma City Thunder": "OKC", "Philadelphia 76ers":"PHI", "Portland Blazers" :"POR", 
                  "Sacramento Kings": "SAC", "Toronto Raptors": "TOR", "Brooklyn Nets": "BRK", "San Antonio Spurs": "SAS", "Utah Jazz": "UTA",
                  "Phoenix Suns":"PHX","Washington Wizards":"WAS"}
        return(team_dict[teamName])
    except ValueError:
        print('Team Abbreviation Not Found!')
    else:
        team_dict = {"Hawks":"ATL", "Celtics":"BOS", "Bulls":"CHI", "Cavaliers":"CLE", "Clippers": "LAC", "Hornets": "CHO", "Mavericks":"DAL", 
                    "Nuggets": "DEN", "Pistons":"DET", "Warriors":"GSW", "Rockets":"HOU", "Lakers": "LAL", "TimberWolves":"MIN", "Pacers": "IND",
                    "Grizzlies": "MEM", "Bucks": "MIL", "Heat":"MIA", "Knicks": "NYK", "Pelicans": "NOP", "Magic": "ORL", "Thunder": "OKC", 
                    "76ers":"PHI", "Blazers" :"POR", "Kings": "SAC", "Raptors": "TOR", "Nets": "NJN", "Spurs": "SAS", "Jazz": "UTA",
                    "Suns":"PHO","Wizards":"WAS", "Atlanta Hawks":"ATL", "Boston Celtics":"BOS", "Chicago Bulls":"CHI", "Cleveland Cavaliers":"CLE", 
                    "Los Angeles Clippers": "LAC", "Charlotte Hornets": "CHO", "Dallas Mavericks":"DAL", 
                    "Denver Nuggets": "DEN", "Detroit Pistons":"DET", "Golden State Warriors":"GSW", "Houston Rockets":"HOU", 
                    "Los Angeles Lakers": "LAL", "Minnesota TimberWolves":"MIN", "Indiana Pacers": "IND",
                    "Memphis Grizzlies": "MEM", "Milwaukee Bucks": "MIL", "Miami Heat":"MIA", "New York Knicks": "NYK", "New Orleans Pelicans": "NOP", 
                    "Orlando Magic": "ORL", "Oklahoma City Thunder": "OKC", "Philadelphia 76ers":"PHI", "Portland Blazers" :"POR", 
                    "Sacramento Kings": "SAC", "Toronto Raptors": "TOR", "Brooklyn Nets": "NJN", "San Antonio Spurs": "SAS", "Utah Jazz": "UTA",
                    "Phoenix Suns":"PHX","Washington Wizards":"WAS"}
        return(team_dict[teamName])


    
    


In [3]:
#### Team Reference Table #### 

def team_ref_table(team):
    try:
        url = 'https://www.basketball-reference.com/teams/' + team_three_letter_name(team) + '/'
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        table = soup.find('table')
        seasons = pd.read_html(str(table))[0]
        del seasons['Unnamed: 8']
        del seasons['Unnamed: 15']
        seasons.columns = ['Season', 'LG', 'Team', 'W', 'L', 'Win %', 'Finish', 'SRS', 'Pace', 'Rel Pace', 'ORtg', 'Rel ORtg', 
                           'Drtg', 'Rel DRtg', 'Playoffs', 'Coaches', 'Top WS']
        seasons['Playoffs'] = seasons['Playoffs'].fillna("Missed")
        seasons['Season'] = seasons['Season'].apply(lambda x: int(str(x)[0:2] + str(x)[-2:]))
        seasons['Team'] = seasons['Team'].apply(lambda x: x.replace("*", ""))
        return(seasons)
    except ValueError:
        print('Could Not Find Team')


In [4]:
##### Team Roster ######
def team_roster(team, season):
    try:
        url = 'https://www.basketball-reference.com/teams/' + team_three_letter_name(team) + '/' + str(season) + '.html' 
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        table = soup.find('table')
        roster = pd.read_html(str(table))[0]
        roster.columns = ['Number', 'Player', 'Position', 'Height', 'Weight', 'Birthday', 'Nationality', 'Years Experience', 'College']
        roster['Birthday'] = roster['Birthday'].apply(lambda x: pd.to_datetime(x))
        roster['Nationality'] = roster['Nationality'].apply(lambda x: x.upper())
        roster['Years Experience'] = roster['Years Experience'].apply(lambda x: 0 if x == 'R' else x)
        roster['Height'] = roster['Height'].apply(lambda x: int(x.split("-")[0]) * 12 + int(x.split("-")[1]))
        return(roster)
    except ValueError:
        print('Could Not Find Roster!')

In [5]:
######## Get Team Results by season ########
def team_results(team, season):
    try:
        url = 'https://www.basketball-reference.com/teams/' + team_three_letter_name(team) + '/' + str(season) + '_games.html' 
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        table = soup.find('table')
        results = pd.read_html(str(table))[0]
        results.columns = ["Game_Number", "Date", "Start Time", "Delete1", "Delete2", "Home/Away", "Oppenent", "Result", 
                          "OverTime", "Points", "Oppenent's Points", "Win Running Total", "Loss Running Total", "Streak", "Notes"]
        results = results[results.Game_Number != 'G']
        del results["Delete1"]
        del results["Delete2"]
        results["Home/Away"] = results["Home/Away"].apply(lambda x: "A" if x == "@" else "H")
        results["Streak"] = results["Streak"].apply(lambda x: -1* int(x.split(" ")[1]) if x.split(" ")[0] == "L" else int(x.split(" ")[1]))
        results["Notes"] = results["Notes"].fillna("None")
        results["Date"] = pd.to_datetime(results["Date"].apply(lambda x: x.split(",")[1]+", "+x.split(",")[2]))
        results["Start Time"] = pd.to_datetime(results["Start Time"].apply(lambda x: str(x) + "m"))
        results["Start Time"] = results["Start Time"].apply(lambda x: x.strftime('%H:%M:%S'))
        results['OverTime'].fillna(0,inplace=True)
        results['OverTime'] = results['OverTime'].apply(lambda x: x if x == 0 else 1 if x == 'OT' else int(x[0]))
        results['Season'] = season
        results['Team'] = team
        return(results)
    except ValueError:
        print("Could Not Find Results For " + team +' '+ str(season))


In [6]:
##### Box Score ######
######  Remove Team Totals for Not FG Data, change plus minus for full game (point difference) #######
def Box_Score(Home, Away, GameDate, Advanced, Team, Period):
    try:
        
        ##### Define for URL ######
        year_url_key = str(datetime.strptime(GameDate, '%m/%d/%Y').year)
        month_url_key = str(datetime.strptime(GameDate, '%m/%d/%Y').month)
        if len(month_url_key) == 1:
            month_url_key = '0'+ month_url_key
        day_url_key = str(datetime.strptime(GameDate, '%m/%d/%Y').day)
        if len(day_url_key) == 1:
            day_url_key = '0'+ day_url_key
        
        url = 'https://www.basketball-reference.com/boxscores/' + year_url_key + month_url_key + day_url_key + '0' + team_three_letter_name(Home) + '.html' 
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        tables = soup.findAll('table')


        ###### Determine Number of OTS #########
        SEASON_OT_CHECK = 0 
        if datetime.strptime(GameDate, '%m/%d/%Y').month in [9,10,11,12]:
            SEASON_OT_CHECK = (int(datetime.strptime(GameDate, '%m/%d/%Y').year) + 1)
        else:
            SEASON_OT_CHECK = datetime.strptime(GameDate, '%m/%d/%Y').year
        
        df_OT_CHECK1 = team_results(Home, SEASON_OT_CHECK)
        df_OT_CHECK2 = df_OT_CHECK1[(df_OT_CHECK1["Date"] == GameDate)]
        Number_of_ots = int(df_OT_CHECK2["OverTime"])


        ###### Away Team Box Score Tables ######## 
        fg_basic_box_score_away_table = tables[0]
        q1_basic_box_score_away_table = tables[1]
        q2_basic_box_score_away_table = tables[2]
        h1_basic_box_score_away_table = tables[3]
        q3_basic_box_score_away_table = tables[4]
        q4_basic_box_score_away_table = tables[5]
        h2_basic_box_score_away_table = tables[6]

        if Number_of_ots != 0:
            OT_dict_basic_box_score_away = {}
            for OT in list(range(1,Number_of_ots+1)):
                OT_dict_basic_box_score_away["ot{0}_basic_box_score_away_table".format(OT)] = tables[6 + OT]

        ###### Away Team Advance Box Score Table ######
        fg_adv_box_score_away_table = tables[7 + Number_of_ots]



        ###### Home Team Box Score Tables ######## 
        fg_basic_box_score_home_table = tables[8 + Number_of_ots]
        q1_basic_box_score_home_table = tables[9 + Number_of_ots]
        q2_basic_box_score_home_table = tables[10 + Number_of_ots]
        h1_basic_box_score_home_table = tables[11 + Number_of_ots]
        q3_basic_box_score_home_table = tables[12 + Number_of_ots]
        q4_basic_box_score_home_table = tables[13 + Number_of_ots]
        h2_basic_box_score_home_table = tables[14 + Number_of_ots]

        if Number_of_ots != 0:
            OT_dict_basic_box_score_home = {}
            for OT in list(range(1,Number_of_ots+1)):
                OT_dict_basic_box_score_home["ot{0}_basic_box_score_home_table".format(OT)] = tables[14 + Number_of_ots + OT]



        ###### Away Team Advance Box Score Tables ######
        fg_adv_box_score_home_table = tables[15 + Number_of_ots*2]



        ####### Dictionary for searching #########
        SEARCH_DICT = {'HOME':  {'BASIC':  {'FG':fg_basic_box_score_away_table  , 'Q1': q1_basic_box_score_away_table , 'Q2':q2_basic_box_score_away_table,
                                            'H1':h1_basic_box_score_away_table, 'Q3':q3_basic_box_score_away_table, 'Q4': q4_basic_box_score_away_table, 
                                            'H2':h2_basic_box_score_away_table }, 
                                'ADV':  {'FG': fg_adv_box_score_away_table }}, 
                       'AWAY': {'BASIC': {'FG': fg_basic_box_score_home_table, 'Q1': q1_basic_box_score_home_table, 'Q2': q2_basic_box_score_home_table,
                                          'H1': fg_basic_box_score_home_table, 'Q3': q3_basic_box_score_home_table, 'Q4': q4_basic_box_score_home_table,
                                          'H2': h2_basic_box_score_home_table }, 
                                'ADV': {'FG': fg_adv_box_score_home_table}}}

        ####### Append Overtime results to search dictionary #########
        if Number_of_ots != 0:
            for OT in list(range(1,Number_of_ots+1)):
                SEARCH_DICT['AWAY']['BASIC']['OT{0}'.format(OT)] = OT_dict_basic_box_score_away["ot{0}_basic_box_score_away_table".format(OT)]
                SEARCH_DICT['HOME']['BASIC']['OT{0}'.format(OT)] = OT_dict_basic_box_score_home["ot{0}_basic_box_score_home_table".format(OT)]


        ############ returner #############
        boxscore = SEARCH_DICT[Team][Advanced][Period]
        boxscore = pd.read_html(str(boxscore))[0]
        boxscore.columns = boxscore.columns.get_level_values(1)
        boxscore = boxscore.rename(columns={'Starters':'Player'})
        boxscore = boxscore[boxscore.Player != 'Reserves']
        boxscore = boxscore.replace('Did Not Play', 0)
        boxscore['Starter/Reserve'] =""
        boxscore.loc[0:4, "Starter/Reserve"] = "Starter"
        boxscore.iloc[-1, boxscore.columns.get_loc('Starter/Reserve')] = 'Team'
        boxscore['Starter/Reserve'] = boxscore['Starter/Reserve'].replace("", "Reserve")
        boxscore[:-1] = boxscore[:-1].fillna(0)
        
        if Period != 'FG':
            boxscore = boxscore[:-1]
            
        
        
        return(boxscore)
    except ValueError:
        print('Could Not Find Box Score!')


In [8]:
############## Create a function to return all games from a certain season to present #############
def Get_model_data(Start_Season, End_Season):
    #### Team List for data ####
    team_list = ["Hawks", "Celtics", "Bulls", "Cavaliers", "Clippers", "Hornets", "Mavericks","Nuggets", "Pistons", "Warriors", "Rockets", "Lakers", "TimberWolves", "Pacers", "Grizzlies", "Bucks", "Heat", "Knicks", "Pelicans", "Magic", "Thunder", "76ers", "Blazers", "Kings", "Raptors", "Nets", "Spurs", "Jazz","Suns","Wizards"]


    ###### Get Years to for loop through ######
    Year_list = list(range(Start_Season, End_Season))

    #### Empty DF ####
    Return_DF = pd.DataFrame({'Game of Season':[], 'Date':[], 'Start Time':[], 'Home/Away': [], 'Oppenent':[], 'Result':[], 'Overtime':[], 'Points':[], "Oppenent's Points":[], 'Win Running Total':[], 'Loss Running Total':[], 'Streak':[], 'Notes':[], 'Season':[], 'Team':[]})

    for season in Year_list:
        for team in team_list:
            #print(team_results(team, season))
            itter_df = team_results(team, season)
            Return_DF.append(itter_df, True)
    return(Return_DF)







In [14]:
team_list = ["Hawks", "Celtics", "Bulls", "Cavaliers", "Clippers", "Hornets", "Mavericks","Nuggets", "Pistons", "Warriors", "Rockets", "Lakers", "TimberWolves", "Pacers", "Grizzlies", "Bucks", "Heat", "Knicks", "Pelicans", "Magic", "Thunder", "76ers", "Blazers", "Kings", "Raptors", "Nets", "Spurs", "Jazz","Suns","Wizards"] 

Year_list = list(range(2010, 2018))

Return_DF = pd.DataFrame({'Game of Season':[], 'Date':[], 'Start Time':[], 'Home/Away': [], 'Oppenent':[], 'Result':[], 'Overtime':[], 'Points':[], "Oppenent's Points":[], 'Win Running Total':[], 'Loss Running Total':[], 'Streak':[], 'Notes':[], 'Season':[], 'Team':[]})

itter_num = 0


for year in Year_list:    
    for team in team_list:
        if itter_num == 0:
            itter_DF = team_results(team, year)
            itter_num = itter_num + 1
        else:
            itter_DF = itter_DF.append(team_results(team, year), ignore_index=True)
            itter_num = itter_num + 1


#practice_DF = team_results("Hawks", 2010)
            
#practice_DF        




Could Not Find Results For Nets 2010
Could Not Find Results For Nets 2011
Could Not Find Results For Nets 2012
Could Not Find Results For Pelicans 2014
Could Not Find Results For Hornets 2015
Could Not Find Results For Pelicans 2015
Could Not Find Results For Hornets 2016


KeyboardInterrupt: 

In [9]:
Get_model_data(2017,2018)

Could Not Find Results For Hornets 2017
Could Not Find Results For Pelicans 2017


,Game of Season,Date,Start Time,Home/Away,Oppenent,Result,Overtime,Points,Oppenent's Points,Win Running Total,Loss Running Total,Streak,Notes,Season,Team


In [30]:
####### Metrics for Models ########
####### will be season totals and trending/MAs #######
### Wins 
### Losses
# Pythagreom wins (Expected Wins based on points scored and allowed)
# Pythagreom Losses (Expcted Losses based on points scored and allowed)
# Margin of victory
# Stregth of schedule (denominated as Points above/below average with 0 being the averge)
#Defensive Rating : Estimate of of points allowed per 100 possesions 
# Pace factor: Estmate of number of possesions in 48 mins
#Free throw attempt rate: Numer of FT attempts per number of FG attempts
#3-point attempt rate: % of FG attempts from 3 Point range
# Effective Field Goal Percentage:This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal
#Turnover Percentag: An estimate of turnovers committed per 100 plays
####### Four Factors
#Offensive Rebound Percentage: An estimate of the percentage of available offensive rebounds a player grabbed while he was on the floor.
#Free Throws Per Field Goal Attempt
# Oppenent Effective Field Goal Attempt
# Oppenent Turnover Percentage
# Defensive Rebound Percentage 
#Oppenent Free Throws Per Field Goal Attempt


,Game_Number,Date,Start Time,Home/Away,Oppenent,Result,OverTime,Points,Oppenent's Points,Win Running Total,Loss Running Total,Streak,Notes,Season,Team
0,1,2009-10-28,19:00:00,H,Indiana Pacers,W,0,120,109,1,0,1,None,2010,Hawks
1,2,2009-10-30,19:30:00,H,Washington Wizards,W,0,100,89,2,0,2,None,2010,Hawks
2,3,2009-11-01,21:30:00,A,Los Angeles Lakers,L,0,110,118,2,1,-1,None,2010,Hawks
3,4,2009-11-03,22:00:00,A,Portland Trail Blazers,W,0,97,91,3,1,1,None,2010,Hawks
4,5,2009-11-04,22:00:00,A,Sacramento Kings,W,0,113,105,4,1,2,None,2010,Hawks
5,6,2009-11-06,19:00:00,A,Charlotte Bobcats,L,0,83,103,4,2,-1,None,2010,Hawks
6,7,2009-11-07,19:00:00,H,Denver Nuggets,W,0,125,100,5,2,1,None,2010,Hawks
7,8,2009-11-11,19:30:00,A,New York Knicks,W,0,114,101,6,2,2,None,2010,Hawks
8,9,2009-11-13,20:00:00,A,Boston Celtics,W,0,97,86,7,2,3,None,2010,Hawks
9,10,2009-11-14,19:00:00,H,New Orleans Hornets,W,0,121,98,8,2,4,None,2010,Hawks


In [ ]:
team_results()